

```
# This is formatted as code
```

1️⃣ Install and setup PySpark

In [ ]:
# ===== Install Java 17 =====
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless -qq
# ===== Set JAVA_HOME =====
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
# ===== Install PySpark =====
!pip install -q pyspark
# ===== Start Spark Session =====
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab3_SparkSQL").getOrCreate()

print("Spark Version:", spark.version)
!java -version


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../openjdk-17-jdk-headless_17.0.17+10-1~22.04_amd64.deb ...
Unpacking openjdk-17-jdk-headless:amd64 (17.0.17+10-1~22.04) over (17.0.16+8~us1-0ubuntu1~22.04.1) ...
Preparing to unpack .../openjdk-17-jre-headless_17.0.17+10-1~22.04_amd64.deb ...
Unpacking openjdk-17-jre-headless:amd64 (17.0.17+10-1~22.04) over (17.0.16+8~us1-0ubuntu1~22.04.1) ...
Setting up openjdk-17-jre-headless:amd64 (17.0.17+10-1~22.04) ...
Installing new version of config file /etc/java-17-openjdk/security/default.policy ...
Installing new version of config file /etc/java-17-openjdk/security/java.security ...
Setting up openjdk-17-jdk-headless:amd64 (17.0.17+10-1~22.04) ...
Spark Version: 3.5.1
openjdk version "17.0.17" 2025-10-21
OpenJDK

2️⃣ Load CSV into a DataFrame

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Start Spark session
spark = SparkSession.builder.appName("Lab3_SparkSQL").getOrCreate()

# Load the CSV with tab separator
df = spark.read.csv("/content/drive/MyDrive/ngram.csv", sep="\t", header=False, inferSchema=True)

# Rename columns
df = df.toDF("ngram", "Year", "Count", "Pages", "Books") \
       .withColumn("Year", col("Year").cast("int")) \
       .withColumn("Count", col("Count").cast("int")) \
       .withColumn("Pages", col("Pages").cast("int")) \
       .withColumn("Books", col("Books").cast("int"))

df.show(20)


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2001|    5|    4|    4|
|! $17.95|2002|    5|    5|    5|
|! $17.95|2003|    2|    2|    2|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|! $17.95|2006|    5|    5|    5|
|! $17.95|2007|    2|    2|    2|
|! $17.95|2008|    2|    2|    2|
+--------+----+-----+-----+-----+
only showing top 20 rows



3️⃣ Create a temporary view for SQL

In [ ]:
df.createOrReplaceTempView("ngrams_table")


**Queries**

1️⃣ Bigram count > 5

In [ ]:
spark.sql("SELECT * FROM ngrams_table WHERE Count > 5").show(5)
df.filter(df.Count > 5).show(5)


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
+--------+----+-----+-----+-----+
only showing top 5 rows

+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
+--------+----+-----+-----+-----+
only showing top 5 rows



2️⃣ Total bigrams per year

In [ ]:
spark.sql("""
    SELECT Year, SUM(Count) as total_bigrams
    FROM ngrams_table
    GROUP BY Year
    ORDER BY Year
""").show()


+----+-------------+
|Year|total_bigrams|
+----+-------------+
|1780|            1|
|1803|            1|
|1806|            1|
|1823|            1|
|1824|            1|
|1825|            1|
|1829|            3|
|1830|            2|
|1831|            1|
|1833|            1|
|1834|            4|
|1836|            1|
|1839|            1|
|1840|            1|
|1841|            2|
|1845|            1|
|1847|            2|
|1848|            1|
|1856|            1|
|1857|            2|
+----+-------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import sum
df.groupBy("Year").agg(sum("Count").alias("total_bigrams")).orderBy("Year").show()


+----+-------------+
|Year|total_bigrams|
+----+-------------+
|1780|            1|
|1803|            1|
|1806|            1|
|1823|            1|
|1824|            1|
|1825|            1|
|1829|            3|
|1830|            2|
|1831|            1|
|1833|            1|
|1834|            4|
|1836|            1|
|1839|            1|
|1840|            1|
|1841|            2|
|1845|            1|
|1847|            2|
|1848|            1|
|1856|            1|
|1857|            2|
+----+-------------+
only showing top 20 rows



3️⃣ Most frequent bigram per year

In [ ]:
spark.sql("""
    SELECT Year, ngram, MAX(Count) as max_count
    FROM ngrams_table
    GROUP BY Year, ngram
    ORDER BY Year
""").show(10)


+----+-----+---------+
|Year|ngram|max_count|
+----+-----+---------+
|1780| ! 09|        1|
|1803| ! 09|        1|
|1806| ! 09|        1|
|1823| ! 09|        1|
|1824| ! 09|        1|
|1825| ! 09|        1|
|1829| ! 09|        3|
|1830| ! 09|        2|
|1831| ! 09|        1|
|1833| ! 09|        1|
+----+-----+---------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import max
df.groupBy("Year", "ngram").agg(max("Count").alias("max_count")).orderBy("Year").show(10)


+----+-----+---------+
|Year|ngram|max_count|
+----+-----+---------+
|1780| ! 09|        1|
|1803| ! 09|        1|
|1806| ! 09|        1|
|1823| ! 09|        1|
|1824| ! 09|        1|
|1825| ! 09|        1|
|1829| ! 09|        3|
|1830| ! 09|        2|
|1831| ! 09|        1|
|1833| ! 09|        1|
+----+-----+---------+
only showing top 10 rows



4️⃣ Bigrams appearing in 20 different years

In [ ]:
spark.sql("""
    SELECT ngram, COUNT(DISTINCT Year) as years_count
    FROM ngrams_table
    GROUP BY ngram
    HAVING years_count = 20
""").show()


+--------+-----------+
|   ngram|years_count|
+--------+-----------+
|! $17.95|         20|
+--------+-----------+



In [ ]:
from pyspark.sql.functions import countDistinct
df.groupBy("ngram").agg(countDistinct("Year").alias("years_count")).filter("years_count = 20").show()


+--------+-----------+
|   ngram|years_count|
+--------+-----------+
|! $17.95|         20|
+--------+-----------+



5️⃣ Bigrams containing '!' in first part and '9' in second part

In [ ]:
spark.sql("""
    SELECT *
    FROM ngrams_table
    WHERE split(ngram, ' ')[0] LIKE '%!%'   -- first part contains '!'
      AND split(ngram, ' ')[1] LIKE '%9%'   -- second part contains '9'
""").show(10)



+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
+--------+----+-----+-----+-----+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import split, col

df.filter(
    (split(col("ngram"), " ")[0].contains("!")) &
    (split(col("ngram"), " ")[1].contains("9"))
).show(50)


+--------+----+-----+-----+-----+
|   ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2001|    5|    4|    4|
|! $17.95|2002|    5|    5|    5|
|! $17.95|2003|    2|    2|    2|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|! $17.95|2006|    5|    5|    5|
|! $17.95|2007|    2|    2|    2|
|! $17.95|2008|    2|    2|    2|
|    ! 09|1780|    1|    1|    1|
|    ! 09|1803|    1|    1|    1|
|    ! 09|1806|    1|    1|    1|
|    ! 09|1823|    1|    1|    1|
|    ! 09|1824|    1|    1|    1|
|    ! 09|1825|    1|    1|    1|
|    ! 09|1829

6️⃣ Bigrams appearing in all years

In [ ]:
total_years = df.select("Year").distinct().count()
spark.sql(f"""
    SELECT ngram, COUNT(DISTINCT Year) as year_count
    FROM ngrams_table
    GROUP BY ngram
    HAVING year_count = {total_years}
""").show()


+-----+----------+
|ngram|year_count|
+-----+----------+
| ! 09|       100|
+-----+----------+



In [ ]:
from pyspark.sql.functions import countDistinct
total_years = df.select("Year").distinct().count()
df.groupBy("ngram").agg(countDistinct("Year").alias("year_count")).filter(f"year_count = {total_years}").show()


+-----+----------+
|ngram|year_count|
+-----+----------+
| ! 09|       100|
+-----+----------+



7️⃣ Total pages and books per bigram per year, sorted alphabetically

In [ ]:
spark.sql("""
    SELECT ngram, Year, SUM(Pages) as total_pages, SUM(Books) as total_books
    FROM ngrams_table
    GROUP BY ngram, Year
    ORDER BY ngram ASC
""").show()


+--------+----+-----------+-----------+
|   ngram|Year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import sum
df.groupBy("ngram", "Year").agg(sum("Pages").alias("total_pages"), sum("Books").alias("total_books")).orderBy("ngram").show()


+--------+----+-----------+-----------+
|   ngram|Year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



8️⃣ Total distinct bigrams per year, sorted by descending year

In [ ]:
spark.sql("""
    SELECT Year, COUNT(DISTINCT ngram) as distinct_bigrams
    FROM ngrams_table
    GROUP BY Year
    ORDER BY Year DESC
""").show()


+----+----------------+
|Year|distinct_bigrams|
+----+----------------+
|2008|               2|
|2007|               2|
|2006|               2|
|2005|               2|
|2004|               2|
|2003|               2|
|2002|               2|
|2001|               2|
|2000|               2|
|1999|               2|
|1998|               2|
|1997|               2|
|1996|               2|
|1995|               2|
|1993|               2|
|1992|               2|
|1991|               2|
|1990|               2|
|1987|               2|
|1985|               2|
+----+----------------+
only showing top 20 rows



In [ ]:
df.groupBy("Year").agg(countDistinct("ngram").alias("distinct_bigrams")).orderBy(df.Year.desc()).show()


+----+----------------+
|Year|distinct_bigrams|
+----+----------------+
|2008|               2|
|2007|               2|
|2006|               2|
|2005|               2|
|2004|               2|
|2003|               2|
|2002|               2|
|2001|               2|
|2000|               2|
|1999|               2|
|1998|               2|
|1997|               2|
|1996|               2|
|1995|               2|
|1993|               2|
|1992|               2|
|1991|               2|
|1990|               2|
|1987|               2|
|1985|               2|
+----+----------------+
only showing top 20 rows

